# 9517 Proj

## 1. Import library

In [172]:
import pandas as pd
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVC, LinearSVC
from sklearn.model_selection import train_test_split
import pickle
import itertools

# need to delete
from skimage import feature as ft

## 2. Define functions to extract features

In [38]:
def get_color_hist(image, nbins, bins_range):
    '''
    Functions to extract histogram features
    '''
    # compute histogram from R, G, B 3-dim
    R_hist = np.histogram(img[:, :, 0], bins = nbins, range = bins_range)
    G_hist = np.histogram(img[:, :, 1], bins = nbins, range = bins_range)
    B_hist = np.histogram(img[:, :, 2], bins = nbins, range = bins_range)
    # concatenate the histograms to a single histogram features
    hist_features = np.concatenate((R_hist[0], G_hist[0], B_hist[0]))
    return hist_features

In [28]:
def get_hog_feature(image):
    '''
    Functions to extract hog features
    '''
    features, hog_image = ft.hog(image, orientations=12, pixels_per_cell=[20, 20], cells_per_block=[2,2], visualize = True, transform_sqrt= False)
    return features, hog_image

In [25]:
def bin_spatial(image, size):
    '''
    Functions to extract origin pixel features
    '''
    features = cv2.resize(image, size).ravel()
    return features

In [199]:
def extract_features(image):
    features = []
    hog_features = []
    # hog feature
    for channel in range(image.shape[2]):
        fea, hog_image = get_hog_feature(image[:, :, channel])
        hog_features.append(fea)
    hog_features = np.ravel(hog_features)
    features.append(hog_features)
    # spatial feature
    features.append(bin_spatial(image, size = (8,8)))
    # histogram feature
    nbins = 32
    bin_range = (0, 256)
    features.append(get_color_hist(image, nbins, bin_range))
    return np.concatenate(features)

## 3. Prepare dataset (train and test)

In [222]:
def get_data(basePath, clips_size, labelType):
    '''
    labelType: 1 as positive -1 as negtive
    '''
    train_data = []
    for index in range(1, clips_size + 1):
        if index % 10 == 0:
            print('processing on', index)
        # store all train image into list
        img_dir = basePath + str(index) + '/imgs/040.jpg'
        # gt_dir = basePath + str(index) + '/annotation.json'
        img = cv2.imread(img_dir)
        if labelType == -1:
            # extract nocar image
            img = img[0:128, 0:256]
            # 1280 * 750
        
        img = cv2.resize(img, (128, 256))
        feature_img = extract_features(img)
        train_data.append(feature_img)
        # store all labels into list
        train_data = list(itertools.chain(train_data))
    return train_data

In [218]:
def get_train_dataset(clips_size):
    basePath = './data/train/clips/'
    car_train_data = get_data(basePath, clips_size, 1)
    noncar_train_data = get_data(basePath, clips_size, -1)
    train_data = np.vstack((car_train_data, noncar_train_data))
    labels = np.hstack((np.ones(len(car_train_data)), np.zeros(len(noncar_train_data))))
    '''
    shape 3 :
    '''
    return train_data, labels

In [223]:
def get_test_dataset(clips_size):
    basePath = './data/test/clips/'
    car_train_data = get_data(basePath, clips_size, 1)
    labels = np.ones(len(car_train_data))
    return np.array(train_data), labels

In [224]:
def get_ground_truth(filepath):
    labels = []
    gt = pd.read_json(filepath)
    list = gt['bbox'].tolist()
    for car in list:
        labels.append(car)
    return labels

## 2. SVM construction

In [225]:
# train_data, train_labels = get_train_dataset(10)
# test_data, test_labels = get_test_dataset(20)
x, y = get_train_dataset(10)
print(x.shape)
print(y.shape)
train_data, test_data, train_labels, test_labels = train_test_split(x,y,test_size = 0.2, random_state=42)

processing on 10
processing on 10


In [213]:
print(train_data.shape)
print(train_labels.shape)

(16, 8208)
(16,)


In [210]:
def svm_train(train_data, train_labels, test_data, test_labels):
    # import svc model
    svc = LinearSVC()
    # fit the data
    # print(train_data.shape)
    # print(train_labels.shape)
    svc.fit(train_data, train_labels)
    with open('model', 'wb') as f:
        pickle.dump(svc, f)
    train_labels = train_labels.reshape(-1, 1)
    test_labels = test_labels.reshape(-1, 1)
    test_acc = svc.score(train_labels, test_labels)
    print('Test Accuracy: {}'.format(round(test_accuracy, 4)))
    # n_predict = 10
    # print('My SVC predicts: ', svc.predict(x_test[0:n_predict]))
    # print('For these',n_predict,'labels: ',y_test[0:n_predict])

    return svc


In [211]:
svm_train(train_data, train_labels, test_data, test_labels)

/opt/anaconda3/envs/COMP9517/lib/python3.6/site-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


ValueError: X has 1 features per sample; expecting 8208